In [0]:
dbutils.widgets.text("p_file_date", "2024-12-30")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
movie_df = spark.read.format("delta").load(f"{silver_folder_path}/movies").filter(f"file_date = '{v_file_date}'")

In [0]:
movie_filtered_df = movie_df.filter(movie_df.year_release_date >= 2010).select(movie_df.movie_id, movie_df.title, movie_df.budget, movie_df.revenue, movie_df.duration_time, movie_df.release_date)

In [0]:
display(movie_filtered_df)

In [0]:
pais_df = spark.read.format("delta").load(f"{silver_folder_path}/country").select("country_id", "country_name")

In [0]:
productora_df = spark.read.format("delta").load(f"{silver_folder_path}/productions_companies").filter(f"file_date = '{v_file_date}'").select("company_id", "company_name")

In [0]:
pais_movie_df = spark.read.format("delta").load(f"{silver_folder_path}/production_country").filter(f"file_date = '{v_file_date}'").select("movie_id", "country_id")
display(pais_movie_df)

In [0]:
movie_company = spark.read.format("delta").load(f"{silver_folder_path}/movie_company").select("movie_id", "company_id")

In [0]:
movie_y_pais_df = movie_filtered_df\
    .join(pais_movie_df, movie_filtered_df.movie_id == pais_movie_df.movie_id, "inner")\
        .select(movie_filtered_df.movie_id, movie_filtered_df.title, movie_filtered_df.budget, movie_filtered_df.release_date, movie_filtered_df.revenue, movie_filtered_df.duration_time, pais_movie_df.country_id)\
        .join(pais_df, pais_movie_df.country_id == pais_df.country_id, "inner")\
            .select(movie_filtered_df.movie_id, movie_filtered_df.title, movie_filtered_df.budget, movie_filtered_df.release_date, movie_filtered_df.revenue, movie_filtered_df.duration_time, pais_df.country_name, pais_df.country_id)
display(movie_y_pais_df)

In [0]:
join_productoras = movie_company\
    .join(productora_df, movie_company.company_id == productora_df.company_id, "inner")\
        .select(movie_company.movie_id, productora_df.company_name, movie_company.company_id)
display(join_productoras)

In [0]:
final_df = movie_y_pais_df\
    .join(join_productoras, movie_y_pais_df.movie_id == join_productoras.movie_id, "inner")\
        .select(join_productoras.movie_id, join_productoras.company_id, movie_y_pais_df.country_id, movie_y_pais_df.title, movie_y_pais_df.budget, movie_y_pais_df.release_date, movie_y_pais_df.revenue, movie_y_pais_df.duration_time, movie_y_pais_df.country_name, join_productoras.company_name)
display(final_df)

In [0]:
from pyspark.sql.functions import current_timestamp

In [0]:
results_df = final_df.withColumn("created_date", current_timestamp())
results_df = results_df.orderBy(results_df.title.asc())
display(results_df)

In [0]:
merge_condition = 'tgt.movie_id = src.movie_id AND tgt.country_id = src.country_id AND tgt.company_id = src.company_id AND tgt.created_date = src.created_date'
merge_delta_lake(results_df, "movie_gold", "results_country_prod_company", gold_folder_path, merge_condition, "created_date")

In [0]:
display(results_df)

In [0]:
display(results_df)

In [0]:
%sql
SELECT * FROM movie_gold.results_country_prod_company

In [0]:
display(spark.read.format("delta").load(f"{gold_folder_path}/results_country_prod_company"))